# Image Classification: MNIST

Models:
+ [Vanilla NODE](#NODE)
+ [Aug.NODE](#ANODE)
+ [Input-Layer](#ILNODE)
+ [2nd-Ordered](#2NODE)

In [1]:
from torchdyn.models import *; from torchdyn import *
from torchdyn.nn import Augmenter

import torch
import torch.nn as nn

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.callbacks import Callback, ModelCheckpoint

from utils import Learner, get_MNIST_dloaders, MetricTracker

## Initialization

In [2]:
# quick run for automated notebook validation
dry_run = False

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


## Data Loading

In [4]:
epoch = 20
path_to_data='./data/mnist_data'

trainloader, testloader = get_MNIST_dloaders(batch_size=64, size=28, path=path_to_data, download=False, num_workers=8)

## Model Dimension Check

In [5]:
# dim = 32
# class Model(nn.Module):
#   def __init__(self):
#     super(Model, self).__init__()
#     self.net = nn.Sequential(
#         # nn.GroupNorm(1,1)
#         ## input data = (64,1,28,28)
#         # nn.Conv2d(1 , dim, 3, padding=1, bias=False),
#         Augmenter(1,0),
#         Augmenter(1,0),
#         # # --- NODE
#         # nn.GroupNorm(dim, dim),
#         nn.Conv2d(1, dim, 3, padding=1, bias=False),
#         nn.Softplus(),                   
#         nn.Conv2d(dim, dim, 3, padding=1, bias=False),
#         nn.Softplus(), 
#         nn.GroupNorm(dim, dim),
#         nn.Conv2d(dim, 1, 1),
#         # # # ---
#         nn.Conv2d(1, 3, 1),
#         nn.AdaptiveAvgPool2d(4),
#         nn.Flatten(),                     
#         nn.Linear(3*16, 10), 
#         )
#   def forward(self, x):
#     for layer in self.net:
#         x = layer(x)
#         print(x.size())
#     return x

# model = Model()
# x = torch.randn(64, 1, 28, 28)

# # Let's print it
# model(x)

<a id = 'NODE'></a>
## Vanilla Neural ODE

+ **vector field $f_\theta$**:  3–layer depth–invariant CNNs,with each layer followed by an instance normalization layer

In [6]:
dim = 42
num_c = 1
vfunc = nn.Sequential(
        nn.Conv2d(1, dim, 3, padding=1, bias=False),
        nn.Softplus(),                  
        nn.Conv2d(dim, dim, 3, padding=1, bias=False),
        nn.Softplus(), 
        nn.GroupNorm(dim, dim),
        nn.Conv2d(dim, 1, 1),
    ).to(device)

In [7]:
nde = NeuralODE(vfunc, 
               solver='dopri5',
               sensitivity='adjoint',
               atol=1e-4,
               rtol=1e-4
               ).to(device)
# NOTE: the first noop `Augmenters` is used only to keep the `nde` at index `2`. Used to extract NFEs in Learner.
model = nn.Sequential(Augmenter(1, 0), # does nothing
                      Augmenter(1, 0), # does nothing
                      nde,
                      nn.Conv2d(1, 3, 1),
                      nn.AdaptiveAvgPool2d(4),
                      nn.Flatten(),                     
                      nn.Linear(3*16, 10)).to(device)

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [8]:
learn = Learner(model, trainloader, testloader, device)
cb1 = MetricTracker()
logger = TensorBoardLogger(save_dir='lightning_logs/MNIST/', name = 'model1')
trainer = pl.Trainer(max_epochs=epoch,
                     accelerator=accelerator,
                     fast_dev_run=dry_run,
                     gpus=torch.cuda.device_count(),
                     logger = logger,
                     callbacks = [cb1]
                     )
trainer.fit(learn)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | model    | Sequential       | 16.9 K
1 | loss     | CrossEntropyLoss | 0     
2 | accuarcy | Accuracy         | 0     
----------------------------------------------
16.9 K    Trainable params
0         Non-trainable params
16.9 K    Total params
0.068     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

C:\Users\m1052\miniconda3\envs\torchenv\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:229: UserWarning: You called `self.log('epoch', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


In [9]:
trainer.test(learn)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9747999906539917
        test_loss           0.08173473924398422
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.08173473924398422, 'test_accuracy': 0.9747999906539917}]

In [10]:
file = './lightning_logs/MNIST/model1/logs.pt'
torch.save(cb1.collection, file)
t = torch.load(file)
t[-2:]

[{'epoch': tensor(19., device='cuda:0'),
  'train_loss': tensor(0.0129, device='cuda:0'),
  'accuracy': tensor(1., device='cuda:0'),
  'NFE': tensor(200., device='cuda:0')},
 {'test_loss': tensor(0.0817, device='cuda:0'),
  'test_accuracy': tensor(0.9748, device='cuda:0')}]